In [1]:
# alwasy get fresh copies
! cp ~cs186/sp16/hw2/part1/part1.db .
! chmod 700 part1.db
! cp ~cs186/sp16/hw2/part1/part1test.db .
! chmod 700 part1test.db

In [2]:
import subprocess, sqlite3, csv, os

# change this to 'part1.db' if you want to test for what data
#   our grading runs on
currentDB = "part1test.db"

# for autograder!
if ('CS186GRADER' in os.environ):
    currentDB = "part1.db"    

if not (os.path.isfile(currentDB)):
    raise IOError("DB File " + currentDB + " is not here!")
db = sqlite3.connect(currentDB)
db.text_factory = str

In [3]:
# Q1
# Question: for all companies whose status column contains 'acquired', show
# the associated acquisition price_amount (if there is no acquisition record, price_amount
# should be returned as NULL)
# Output: company_name, price_amount

def q1():
    db.executescript("""
    DROP VIEW IF EXISTS q1;
    
    CREATE VIEW q1 AS
        SELECT c.company_name, a.price_amount
        FROM companies c 
        LEFT JOIN acquisitions a
        ON c.company_name = a.company_name
        WHERE c.status LIKE '%acquired%';
    """)
    return
q1()

In [4]:
# Q2
# Question: How many startups, according this data, were founded (founded_at column) between 
# 2012 and 2014, inclusive (on or after 2012-01-01, and before or on 2014-12-31)?
# Output: a number (column name doesn't matter)
# Hint: time comparison will look like <= '1990-01-12'

def q2():
    db.executescript("""
    DROP VIEW IF EXISTS q2;
    
    CREATE VIEW q2 AS
        SELECT count(*) AS total
        FROM companies
        WHERE founded_at >= '2012-01-01' AND
        founded_at <= '2014-12-31';
    """)
    return

q2() # called for your convenience

In [5]:
# Q3
# Question: What is/are the state(s) that has/have the largest number(s) of startups  in the  
#    "Security" market (i.e. market column contains the word "Security")? (Plural due to ties)
# Output: the state(s) and the corresponding number(s), column named "state" and "total" respectively.
#         Note that the test script relies on the naming of output columns.

# Notes:
# - If the output is tied, then display all (i.e. if both CA and NY have, say 1000 startups, 
#   then display both states)
# - You might want to look into the keyword "LIKE"
# - The state must be valid (i.e. not "")

# company names must be distinct?
def q3():
    db.executescript("""
    DROP VIEW IF EXISTS q3;
    
    CREATE VIEW q3 AS
        SELECT state, num as total FROM 
            (SELECT b.state, count(*) AS num
                FROM companies b
                WHERE market LIKE '%Security%'
                GROUP BY b.state) h
                WHERE h.num = (
                    SELECT MAX(f.num) FROM
                    (SELECT state, count(*) AS num
                        FROM companies
                        WHERE market LIKE '%Security%'
                        GROUP BY state) f);
    """)
    return

q3()

In [6]:
# Q4
# Question: which cities have a larger number of acquirers than startups?

# Note 
# - a startup could be counted as an acquirer, 
#   but it should only be counted at most once in each category
#   and the data is not very clean (i.e. duplicates in company_name entry)

def q4():
    db.executescript("""
    DROP VIEW IF EXISTS q4;
    CREATE VIEW q4 AS
        SELECT aa.acquirer_city
        
        FROM 
        (SELECT acquirer_city, count(DISTINCT acquirer_name) AS acq_num
            FROM acquisitions 
            GROUP BY acquirer_city) aa   
        LEFT JOIN 
        (SELECT city, count(DISTINCT company_name) As comp_num
            FROM companies
            GROUP BY city) cc 
            
        ON cc.city = aa.acquirer_city
        WHERE aa.acq_num > cc.comp_num OR
        cc.comp_num IS NULL;

    """)
    return
q4()

In [7]:
q1()
q2()
q3()
q4()
! ./test.sh

PASS q1
PASS q2
PASS q3
PASS q4
SUCCESS: Your queries worked on this dataset!
